In [4]:
import pandas as pd
import re
from collections import defaultdict

In [ ]:
## Uun this code when using it for the first time for loading the dataset
# pip install huggingface_hub

In [2]:
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv")

C:\Users\liu.xuanc\AppData\Local\Temp\ipykernel_17284\2761146993.py:1: DtypeWarning: Columns (2,3,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv")


In [14]:
# def clean_chords(chord_string):
#     """
#     Clean chord strings, remove structural labels (such as<intro_1>,<verse_1>, etc.),，
    
#     Parameters:
#         Chord_string: A string containing labels and chords
        
#     return:
#         Chords progression: List
#     """
#     if pd.isna(chord_string) or chord_string == '':
#         return []
    
#     # delete <...> lable
#     cleaned = re.sub(r'<[^>]+>', '', chord_string)
    
#     # delete other string
#     chords = [chord.strip() for chord in cleaned.split() if chord.strip()]
    
#     return chords

In [37]:
NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# A simple homophone conversion dictionary (for standardized input)
FLAT_TO_SHARP = {
    'Db': 'C#', 'Eb': 'D#', 'Gb': 'F#', 'Ab': 'G#', 'Bb': 'A#',
    'db': 'C#', 'eb': 'D#', 'gb': 'F#', 'ab': 'G#', 'bb': 'A#'
}

In [11]:
# ==========================================
# Global configuration and auxiliary functions
# ==========================================

def parse_chord_root_quality(chord_str):
    """
    Auxiliary function: parses a single chord string and returns (Root, Quality)
    For example: "Am" ->("A", "min"), "G7" ->("G", "7")
    """
    chord_str = chord_str.strip()
    if not chord_str: return None, None
    
    for flat, sharp in FLAT_TO_SHARP.items():
        if chord_str.startswith(flat):
            chord_str = sharp + chord_str[len(flat):]
            break
    if len(chord_str) > 1 and chord_str[1] == 's': # Fs7
         chord_str = chord_str[0] + '#' + chord_str[2:]
         
    match = re.match(r'([A-G]#?)(.*)', chord_str)
    if not match: return None, None
    
    root = match.group(1)
    rest = match.group(2).lower()
    
    # Characteristic
    if 'dim' in rest: quality = 'dim'
    elif 'min' in rest or 'm' == rest: quality = 'min'
    elif '7' in rest and 'maj' not in rest and 'min' not in rest: quality = '7'
    else: quality = 'maj' # Default to major triad/major seventh chord
    
    return root, quality

In [12]:
# ==========================================
# Segmentation
# ==========================================

def segment_music_string(raw_string):
    """
    Input: '<intro_1> C D <verse_1> F G'
    Output: [{'label': 'intro_1', 'chords': ['C', 'D']}, {'label': 'verse_1', 'chords': ['F', 'G']}]
    """
    parts = re.split(r'(<[^>]+>)', raw_string)
    
    segments = []
    current_label = "Unknown"
    
    for part in parts:
        part = part.strip()
        if not part: continue
        
        # Label (< xxx，> )
        if part.startswith('<') and part.endswith('>'):
            current_label = part.strip('<>')
        else:
            # Content（Chords）
            chord_list = part.split()
            if chord_list:
                segments.append({
                    'label': current_label,
                    'chords': chord_list
                })
                
    return segments

In [13]:
# ==========================================
# Key Inference
# ==========================================

KEY_TEMPLATES = {}
def _build_templates():
    for i in range(12):
        root = NOTES[i]
        # Major key template
        s_maj = [NOTES[(i + n) % 12] for n in [0, 2, 4, 5, 7, 9, 11]]
        maj_chords = {s_maj[0]:['maj'], s_maj[1]:['min'], s_maj[2]:['min'], s_maj[3]:['maj'], s_maj[4]:['maj','7'], s_maj[5]:['min'], s_maj[6]:['dim']}
        KEY_TEMPLATES[f"{root} Major"] = (maj_chords, s_maj[0], s_maj[4]) # (Template, main tone, subordinate tone)
        
        # Minor Tune Template (Natural+Harmony)
        s_min = [NOTES[(i + n) % 12] for n in [0, 2, 3, 5, 7, 8, 10]]
        dom_root = NOTES[(i + 7) % 12]
        min_chords = {s_min[0]:['min'], s_min[1]:['dim'], s_min[2]:['maj'], s_min[3]:['min'], s_min[4]:['min'], dom_root:['maj','7'], s_min[5]:['maj'], s_min[6]:['maj','7']}
        KEY_TEMPLATES[f"{root} Minor"] = (min_chords, s_min[0], dom_root)
_build_templates()

def infer_key_from_list(chord_list):
    """
    input: ['C', 'F', 'G7', 'C']
    output: 'C Major'
    """
    if not chord_list: return "Unknown"
    
    parsed_data = []
    for idx, c_str in enumerate(chord_list):
        r, q = parse_chord_root_quality(c_str)
        if r:
            next_r = None
            if idx + 1 < len(chord_list):
                next_r, _ = parse_chord_root_quality(chord_list[idx+1])
            parsed_data.append((r, q, next_r))
            
    scores = defaultdict(int)
    
    # calculate the score for each
    for key_name, (template, tonic, dom) in KEY_TEMPLATES.items():
        score = 0
        for root, quality, next_root in parsed_data:
            # Basic mathc
            if root in template:
                score += 1
                if quality in template[root]:
                    score += 2
            else:
                score -= 1
            
            if root == dom and quality == '7' and next_root == tonic:
                score += 5
                
            if root == tonic:
                score += 1
                
        scores[key_name] = score
        
    if not scores: return "Unknown"
    return max(scores, key=scores.get)

In [17]:
def patch_short_segments(analyzed_segments, min_chords=2):
    """
    Parameters:
    analyzed_segments: list, {'label':..., 'chords':..., 'key':...}
    min_chords: threshold，chords less than this quantity will be corrected (default is 2)
    """
    
    # Scan twice
    
    # orward Pass
    for i in range(1, len(analyzed_segments)):
        current_seg = analyzed_segments[i]
        prev_seg = analyzed_segments[i-1]
        
        if len(current_seg['chords']) < min_chords:
            if prev_seg['key'] != "Unknown":
                current_seg['key'] = prev_seg['key']
                current_seg['key_source'] = 'borrowed_prev' 

    # Backward Pass
    for i in range(len(analyzed_segments) - 2, -1, -1):
        current_seg = analyzed_segments[i]
        next_seg = analyzed_segments[i+1]
        
        if len(current_seg['chords']) < min_chords:
            # (Usually Intro should follow Verse's tone)
            if next_seg['key'] != "Unknown":
                 current_seg['key'] = next_seg['key']
                 current_seg['key_source'] = 'borrowed_next'
                 
    return analyzed_segments

In [14]:
# ==========================================
# Roman Numeral Conversion
# ==========================================
ROMAN_MAP = {
    0: 'I', 1: 'bII', 2: 'II', 3: 'bIII', 4: 'III', 5: 'IV', 
    6: 'bV', 7: 'V', 8: 'bVI', 9: 'VI', 10: 'bVII', 11: 'VII'
}

def convert_to_roman(chord_list, key_str):
    """
    input: chord_list=['C', 'F', 'G'], key_str='C Major'
    output: ['I', 'IV', 'V']
    """
    if key_str == "Unknown":
        return chord_list
    
    # Analyze the main tone of tonality
    key_root_str = key_str.split()[0] # "C Major" -> "C"
    is_minor_key = "Minor" in key_str
    
    if key_root_str not in NOTES: return chord_list
    key_root_idx = NOTES.index(key_root_str)
    
    roman_output = []
    
    for chord_str in chord_list:
        root, quality = parse_chord_root_quality(chord_str)
        if not root:
            roman_output.append("?")
            continue
            
        # Calculate Interval
        # (Root note of chord - tonic tonic) % 12
        root_idx = NOTES.index(root)
        interval = (root_idx - key_root_idx) % 12
        
        base_roman = ROMAN_MAP.get(interval, "?")
        
        # Minor triad (min) or subtract triad (dim) -> Lowercase
        if quality == 'min' or quality == 'dim':
            final_roman = base_roman.lower()
        else:
            final_roman = base_roman
            
        # Suffix
        if quality == '7':
            final_roman += '7'
        elif quality == 'dim':
            final_roman += '°'
            
        roman_output.append(final_roman)
        
    return roman_output

In [15]:
df['chords'][0]

'<intro_1> C <verse_1> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <verse_2> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <chorus_1> F C F C G C F C E7 Amin C F G7 C <solo_1> D <chorus_2> G D G D A D G D Fs7 Bmin D G A7 D G A7 D'

In [16]:
segments = segment_music_string(df['chords'][0])
segments

[{'label': 'intro_1', 'chords': ['C']},
 {'label': 'verse_1',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'verse_2',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'chorus_1',
  'chords': ['F',
   'C',
   'F',
   'C',
   'G',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C']},
 {'label': 'solo_1', 'chords': ['D']},
 {'label': 'chorus_2',
  'chords': ['G',
   'D',
   'G',
   'D',
   'A',
   'D',
   'G',
   'D',
   'Fs7',
   'Bmin',
   'D',
   'G',
   'A7',
   'D',
   'G',
   'A7',
   'D']}]

In [18]:
analyzed_records = []
for seg in segments:
    inferred_key = infer_key_from_list(seg['chords'])
    analyzed_records.append({
        'label': seg['label'],
        'chords': seg['chords'],
        'key': inferred_key
    })

In [19]:
analyzed_records

[{'label': 'intro_1', 'chords': ['C'], 'key': 'C Major'},
 {'label': 'verse_1',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'verse_2',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'chorus_1',
  'chords': ['F',
   'C',
   'F',
   'C',
   'G',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'solo_1', 'chords': ['D'], 'key': 'D Major'},
 {'label': 'chorus_2',
  'chords': ['G',
   'D',
   'G',
   'D',
   'A',
   'D',
   'G',
   'D',
   'Fs7',
   'Bmin',
   'D',
   'G',
   'A7',
   'D',
   'G',
   'A7',
   'D'],
  'key': 'D Major'}]

In [20]:
analyzed_records = patch_short_segments(analyzed_records, min_chords=2)
analyzed_records

[{'label': 'intro_1',
  'chords': ['C'],
  'key': 'C Major',
  'key_source': 'borrowed_next'},
 {'label': 'verse_1',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'verse_2',
  'chords': ['F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'C',
   'G7',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'chorus_1',
  'chords': ['F',
   'C',
   'F',
   'C',
   'G',
   'C',
   'F',
   'C',
   'E7',
   'Amin',
   'C',
   'F',
   'G7',
   'C'],
  'key': 'C Major'},
 {'label': 'solo_1',
  'chords': ['D'],
  'key': 'D Major',
  'key_source': 'borrowed_next'},
 {'label': 'chorus_2',
  'chords': ['G',
   'D',
   'G',
   'D',
   'A',
   'D',
   'G',
   'D',
   'Fs7',
   'Bmin',
   'D',
   'G',
   'A7',
   'D',
   'G',
   'A7',
   'D'],
  'key': 'D Major'}]

In [39]:
chords_output = []
for record in analyzed_records:
    roman = convert_to_roman(record['chords'], record['key'])

    chords_output.append({
        'section': record['label'],
        'key': record['key'],
        'roman': roman
    })

In [40]:
chords_output

[{'section': 'intro_1', 'key': 'C Major', 'roman': ['I']},
 {'section': 'verse_1',
  'key': 'C Major',
  'roman': ['IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'I',
   'V7',
   'I',
   'IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'V7',
   'I']},
 {'section': 'verse_2',
  'key': 'C Major',
  'roman': ['IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'I',
   'V7',
   'I',
   'IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'V7',
   'I']},
 {'section': 'chorus_1',
  'key': 'C Major',
  'roman': ['IV',
   'I',
   'IV',
   'I',
   'V',
   'I',
   'IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'V7',
   'I']},
 {'section': 'solo_1', 'key': 'D Major', 'roman': ['I']},
 {'section': 'chorus_2',
  'key': 'D Major',
  'roman': ['IV',
   'I',
   'IV',
   'I',
   'V',
   'I',
   'IV',
   'I',
   'III7',
   'vi',
   'I',
   'IV',
   'V7',
   'I',
   'IV',
   'V7',
   'I']}]

In [41]:
def extract_and_format_roman(analyzed_records, include_tags=True):
    """
    Parameters:
    analyzed_records: chords_output in the former step
    include_tags: 
                  - True: Keep "<intro> I IV <verse> I V" (Structure + Sequence)
                  - False: Only "I IV I V" (Sequence)
    
    Return:
    str: Roman numerals string
    """
    output_parts = []
    
    for record in analyzed_records:
        roman_seq = " ".join(record['roman'])
        
        if include_tags:
            # "<label> roman_seq"
            segment_str = f"<{record['section']}> {roman_seq}"
            output_parts.append(segment_str)
        else:
            # Roman numerals
            output_parts.append(roman_seq)
            
    return " ".join(output_parts)

In [35]:
final_string_tagged = extract_and_format_roman(chords_output, include_tags=False)

In [42]:
print(extract_and_format_roman(chords_output, include_tags=False))

I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I IV I IV I V I IV I III7 vi I IV V7 I I IV I IV I V I IV I III7 vi I IV V7 I IV V7 I


In [43]:
print(extract_and_format_roman(chords_output, include_tags=True))

<intro_1> I <verse_1> IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I <verse_2> IV I III7 vi I IV I V7 I IV I III7 vi I IV V7 I <chorus_1> IV I IV I V I IV I III7 vi I IV V7 I <solo_1> I <chorus_2> IV I IV I V I IV I III7 vi I IV V7 I IV V7 I
